In [8]:
# notebooks/03_database_storage.ipynb

import os
import pandas as pd
import oracledb
import uuid # Import uuid for generating unique IDs

# --- Global Setup ---
# Define paths relative to the project root.
# Assuming your Jupyter Notebook is in 'week_02_challenge/notebooks/'
# and your data/output folders are directly under 'week_02_challenge/'

# Get the current working directory (where the notebook is run from)
current_notebook_dir = os.getcwd()

# Go up one level to reach the 'week_02_challenge' project root
project_root = os.path.abspath(os.path.join(current_notebook_dir, os.pardir))

output_image_dir = os.path.abspath(os.path.join(project_root, 'output', 'images'))
processed_data_dir = os.path.abspath(os.path.join(project_root, 'data', 'processed'))

os.makedirs(output_image_dir, exist_ok=True)
print(f"Ensured output image directory exists: {output_image_dir}")
os.makedirs(processed_data_dir, exist_ok=True)
print(f"Ensured processed data directory exists: {processed_data_dir}")

# --- 1. Database Configuration (using environment variables) ---
DB_USER = os.getenv('DB_USER', 'system')
DB_PASSWORD = os.getenv('DB_PASSWORD', '123456') # !! IMPORTANT: Replace '123456' with your actual SYSTEM user password !!
DB_HOST = os.getenv('DB_HOST', 'localhost')
DB_PORT = os.getenv('DB_PORT', '1521')
DB_SERVICE_NAME = os.getenv('DB_SERVICE_NAME', 'XEPDB1') # Common for Oracle XE, replace with your actual service name or SID

# Construct the DSN (Data Source Name)
DB_DSN = f"{DB_HOST}:{DB_PORT}/{DB_SERVICE_NAME}"
print(f"\nAttempting to connect to DSN: {DB_DSN} with user: {DB_USER}")


# --- 2. Data Loading (from your previous script) ---
input_filepath = os.path.join(processed_data_dir, 'fintech_app_reviews_analyzed.csv')

df_analyzed = pd.DataFrame() # Initialize empty DataFrame
if not os.path.exists(input_filepath):
    print(f"Error: Analyzed data file not found at {input_filepath}.")
    print("Please ensure your data processing step (e.g., '02_sentiment_thematic_analysis.ipynb') has been run successfully to create this file.")
else:
    df_analyzed = pd.read_csv(input_filepath)
    print(f"Loaded {len(df_analyzed)} analyzed reviews from {input_filepath}")
    print("Columns in df_analyzed:", df_analyzed.columns.tolist())


# --- 3. Database Connection Function ---
def get_db_connection():
    """Establishes and returns a connection to the Oracle database."""
    connection = None
    try:
        connection = oracledb.connect(user=DB_USER, password=DB_PASSWORD, dsn=DB_DSN)
        print("Successfully connected to Oracle Database!")
        return connection
    except oracledb.Error as e:
        error_obj, = e.args
        print(f"Database connection error: {error_obj.message}")
        print("Help: https://docs.oracle.com/error-help/db/ora-00001/")
        print("Failed to connect to Oracle Database. Check config and connection details.")
        return None

# --- 4. Function to Create REVIEWS Table ---
def create_reviews_table(connection):
    """
    Creates the FINTECH_APP_REVIEWS table if it does not already exist.
    Matches the schema of your df_analyzed DataFrame.
    """
    if connection is None:
        print("No database connection available to create REVIEWS table.")
        return False

    cursor = connection.cursor()
    table_name = "FINTECH_APP_REVIEWS"

    create_table_sql = f"""
    CREATE TABLE {table_name} (
        REVIEW_ID VARCHAR2(50) PRIMARY KEY,
        USER_NAME VARCHAR2(255),
        RATING NUMBER(1,0),
        REVIEW_DATE DATE,
        REVIEW_TEXT CLOB,
        BANK_APP_NAME VARCHAR2(255),
        SOURCE VARCHAR2(100),
        SENTIMENT VARCHAR2(50),
        SENTIMENT_SCORE NUMBER(5,4),
        PROCESSED_REVIEWS_TOKENS CLOB,
        EXTRACTED_KEYWORDS CLOB,
        IDENTIFIED_THEME VARCHAR2(255)
        -- Consider adding a foreign key to BANKS.BANK_ID here if needed for strict relational integrity
    )
    """

    try:
        cursor.execute(f"SELECT table_name FROM user_tables WHERE table_name = '{table_name.upper()}'")
        if cursor.fetchone():
            print(f"Table '{table_name}' already exists. Skipping creation.")
            return True
        else:
            print(f"Attempting to create table '{table_name}'...")
            cursor.execute(create_table_sql)
            print(f"Table '{table_name}' created successfully.")
            connection.commit()
            return True
    except oracledb.Error as e:
        error_obj, = e.args
        print(f"Error creating table '{table_name}': {error_obj.message}")
        return False
    finally:
        cursor.close()

# --- 5. Function to Create BANKS Table ---
def create_banks_table(connection):
    """
    Creates the BANKS table if it does not already exist.
    """
    if connection is None:
        print("No database connection available to create BANKS table.")
        return False

    cursor = connection.cursor()
    table_name = "BANKS"

    create_table_sql = f"""
    CREATE TABLE {table_name} (
        BANK_ID VARCHAR2(50) PRIMARY KEY,
        BANK_NAME VARCHAR2(255) UNIQUE NOT NULL
    )
    """

    try:
        cursor.execute(f"SELECT table_name FROM user_tables WHERE table_name = '{table_name.upper()}'")
        if cursor.fetchone():
            print(f"Table '{table_name}' already exists. Skipping creation.")
            return True
        else:
            print(f"Attempting to create table '{table_name}'...")
            cursor.execute(create_table_sql)
            print(f"Table '{table_name}' created successfully.")
            connection.commit()
            return True
    except oracledb.Error as e:
        error_obj, = e.args
        print(f"Error creating table '{table_name}': {error_obj.message}")
        return False
    finally:
        cursor.close()

# --- 6. Function to Insert REVIEWS Data ---
def insert_reviews_data(connection, df):
    """
    Inserts data from the pandas DataFrame into the FINTECH_APP_REVIEWS table.
    Uses executemany for efficient bulk insertion.
    Includes TRUNCATE TABLE to clear data before inserting new, avoiding PK violations.
    """
    if connection is None or df.empty:
        print("No database connection or empty DataFrame to insert REVIEWS data.")
        return

    cursor = connection.cursor()
    table_name = "FINTECH_APP_REVIEWS"

    try:
        print(f"Truncating table '{table_name}' to clear existing data...")
        cursor.execute(f"TRUNCATE TABLE {table_name}")
        connection.commit()
        print(f"Table '{table_name}' truncated.")

        insert_sql = f"""
        INSERT INTO {table_name} (
            REVIEW_ID, USER_NAME, RATING, REVIEW_DATE, REVIEW_TEXT,
            BANK_APP_NAME, SOURCE, SENTIMENT, SENTIMENT_SCORE,
            PROCESSED_REVIEWS_TOKENS, EXTRACTED_KEYWORDS, IDENTIFIED_THEME
        ) VALUES (
            :review_id, :user_name, :rating, :review_date, :review_text,
            :bank_app_name, :source, :sentiment, :sentiment_score,
            :processed_reviews_tokens, :extracted_keywords, :identified_theme
        )
        """

        data_to_insert = []
        for index, row in df.iterrows():
            data_to_insert.append({
                "review_id": str(row['reviewId']) if pd.notna(row['reviewId']) else None,
                "user_name": str(row['User Name']) if pd.notna(row['User Name']) else None,
                "rating": int(row['Rating']) if pd.notna(row['Rating']) else None,
                "review_date": pd.to_datetime(row['Date']).date() if pd.notna(row['Date']) else None,
                "review_text": str(row['Review Text']) if pd.notna(row['Review Text']) else None,
                "bank_app_name": str(row['Bank/App Name']) if pd.notna(row['Bank/App Name']) else None,
                "source": str(row['Source']) if pd.notna(row['Source']) else None,
                "sentiment": str(row['Sentiment']) if pd.notna(row['Sentiment']) else None,
                "sentiment_score": float(row['Sentiment_Score']) if pd.notna(row['Sentiment_Score']) else None,
                "processed_reviews_tokens": str(row['Processed_Reviews_Tokens']) if pd.notna(row['Processed_Reviews_Tokens']) else None,
                "extracted_keywords": str(row['Extracted_Keywords']) if pd.notna(row['Extracted_Keywords']) else None,
                "identified_theme": str(row['Identified_Theme']) if pd.notna(row['Identified_Theme']) else None,
            })

        cursor.executemany(insert_sql, data_to_insert)
        connection.commit()
        print(f"Successfully inserted {cursor.rowcount} rows into '{table_name}'.")
    except oracledb.Error as e:
        error_obj, = e.args
        print(f"Error inserting data into '{table_name}': {error_obj.message}")
        connection.rollback()
    finally:
        cursor.close()

# --- 7. Function to Insert BANKS Data ---
def insert_banks_data(connection, df_reviews):
    """
    Extracts unique bank names from the reviews DataFrame and inserts them
    into the BANKS table. Truncates the BANKS table first.
    """
    if connection is None or df_reviews.empty:
        print("No database connection or empty DataFrame to insert BANKS data.")
        return

    cursor = connection.cursor()
    table_name = "BANKS"

    try:
        print(f"Truncating table '{table_name}' to clear existing data...")
        cursor.execute(f"TRUNCATE TABLE {table_name}")
        connection.commit()
        print(f"Table '{table_name}' truncated.")

        # Get unique bank names
        unique_bank_names = df_reviews['Bank/App Name'].dropna().unique().tolist()
        print(f"Found {len(unique_bank_names)} unique bank names.")

        insert_sql = f"""
        INSERT INTO {table_name} (BANK_ID, BANK_NAME)
        VALUES (:bank_id, :bank_name)
        """

        data_to_insert = []
        for bank_name in unique_bank_names:
            data_to_insert.append({
                "bank_id": str(uuid.uuid4()), # Generate a unique ID for each bank
                "bank_name": str(bank_name)
            })

        if data_to_insert:
            cursor.executemany(insert_sql, data_to_insert)
            connection.commit()
            print(f"Successfully inserted {cursor.rowcount} rows into '{table_name}'.")
        else:
            print("No unique bank names to insert.")

    except oracledb.Error as e:
        error_obj, = e.args
        print(f"Error inserting data into '{table_name}': {error_obj.message}")
        connection.rollback()
    finally:
        cursor.close()

# --- 8. Function to Read Data (Example) ---
def fetch_reviews_data(connection, limit=5):
    """
    Fetches a limited number of reviews from the FINTECH_APP_REVIEWS table.
    """
    if connection is None:
        print("No database connection available to fetch REVIEWS data.")
        return pd.DataFrame()

    cursor = connection.cursor()
    table_name = "FINTECH_APP_REVIEWS"
    select_sql = f"SELECT * FROM {table_name} WHERE ROWNUM <= :limit_val"

    try:
        cursor.execute(select_sql, limit_val=limit)
        columns = [col[0] for col in cursor.description]
        rows = cursor.fetchall()
        df_from_db = pd.DataFrame(rows, columns=columns)
        print(f"Successfully fetched {len(df_from_db)} rows from '{table_name}'.")
        return df_from_db
    except oracledb.Error as e:
        error_obj, = e.args
        print(f"Error fetching data from '{table_name}': {error_obj.message}")
        print("This error fetching might imply the table still isn't visible, or connection issues.")
        return pd.DataFrame()
    finally:
        cursor.close()

def fetch_banks_data(connection, limit=5):
    """
    Fetches a limited number of banks from the BANKS table.
    """
    if connection is None:
        print("No database connection available to fetch BANKS data.")
        return pd.DataFrame()

    cursor = connection.cursor()
    table_name = "BANKS"
    select_sql = f"SELECT * FROM {table_name} WHERE ROWNUM <= :limit_val"

    try:
        cursor.execute(select_sql, limit_val=limit)
        columns = [col[0] for col in cursor.description]
        rows = cursor.fetchall()
        df_from_db = pd.DataFrame(rows, columns=columns)
        print(f"Successfully fetched {len(df_from_db)} rows from '{table_name}'.")
        return df_from_db
    except oracledb.Error as e:
        error_obj, = e.args
        print(f"Error fetching data from '{table_name}': {error_obj.message}")
        return pd.DataFrame()
    finally:
        cursor.close()

# --- Main Execution Block for Database Operations ---
if not df_analyzed.empty:
    db_connection = None
    try:
        db_connection = get_db_connection()
        if db_connection:
            # --- CONNECTION DIAGNOSTICS ---
            print(f"Python connected as user: {db_connection.username}")
            print(f"Python connected to DSN: {db_connection.dsn}")
            cursor_info = db_connection.cursor()
            cursor_info.execute("SELECT SYS_CONTEXT('USERENV', 'CON_NAME') AS PDB_NAME, SYS_CONTEXT('USERENV', 'DB_NAME') AS DB_NAME, SYS_CONTEXT('USERENV', 'INSTANCE_NAME') AS INSTANCE_NAME FROM DUAL")
            db_info = cursor_info.fetchone()
            cursor_info.close()
            if db_info:
                print(f"*** Python connected to Instance Details: PDB={db_info[0]}, DB_NAME={db_info[1]}, INSTANCE={db_info[2]} ***")
            # --- END CONNECTION DIAGNOSTICS ---

            # --- PROCESS BANKS TABLE FIRST ---
            if create_banks_table(db_connection):
                insert_banks_data(db_connection, df_analyzed)
                # Optional: Fetch some data to verify banks
                fetched_banks_df = fetch_banks_data(db_connection)
                if not fetched_banks_df.empty:
                    print("\nSample data fetched from BANKS table:")
                    print(fetched_banks_df.head())
            else:
                print("Skipping BANKS table population due to creation failure.")

            # --- THEN PROCESS REVIEWS TABLE ---
            if create_reviews_table(db_connection):
                insert_reviews_data(db_connection, df_analyzed)
                # Optional: Fetch some data to verify reviews
                fetched_reviews_df = fetch_reviews_data(db_connection)
                if not fetched_reviews_df.empty:
                    print("\nSample data fetched from FINTECH_APP_REVIEWS table:")
                    print(fetched_reviews_df.head())
            else:
                print("Skipping REVIEWS table population due to creation failure.")

    finally:
        if db_connection:
            db_connection.close()
            print("Database connection closed.")
else:
    print("DataFrame for database operations is empty. Skipping database connection.")

Ensured output image directory exists: d:\10academy\10acadamey\week_02_challenge\output\images
Ensured processed data directory exists: d:\10academy\10acadamey\week_02_challenge\data\processed

Attempting to connect to DSN: localhost:1521/XEPDB1 with user: system
Loaded 8986 analyzed reviews from d:\10academy\10acadamey\week_02_challenge\data\processed\fintech_app_reviews_analyzed.csv
Columns in df_analyzed: ['reviewId', 'User Name', 'Rating', 'Date', 'Review Text', 'Bank/App Name', 'Source', 'Sentiment', 'Sentiment_Score', 'Processed_Reviews_Tokens', 'Extracted_Keywords', 'Identified_Theme']
Successfully connected to Oracle Database!
Python connected as user: system
Python connected to DSN: localhost:1521/XEPDB1
*** Python connected to Instance Details: PDB=XEPDB1, DB_NAME=XEPDB1, INSTANCE=xe ***
Attempting to create table 'BANKS'...
Table 'BANKS' created successfully.
Truncating table 'BANKS' to clear existing data...
Table 'BANKS' truncated.
Found 3 unique bank names.
Successfully i